# 一、 特征可视化

In [ ]:
import os
import numpy as np
import torch
from src.models.video_model import VideoCompressor
from skimage import io
os.environ["CUDA_VISIBLE_DEVICES"]="1"

device = torch.device('cpu')
model = VideoCompressor()
new_state_dict = torch.load('output/codec-10-01_14-34-31/epoch_0000_step008000.pt', map_location='cpu')
new_state_dict_ = {}
for name,param in new_state_dict.items():
    new_state_dict_[name.replace('module.', '')] = param
model.load_state_dict(new_state_dict_, strict=True)
model.eval()
model = model.to(device)
rate_idx = 3
p_frame_y_q_scales = [1.2383, 0.9623, 0.7134, 0.5319]
p_frame_mv_y_q_scales = [1.1844, 1.1044, 1.0107, 0.9189]

def to_tensor(img_path):
    img = io.imread(img_path)
    img = torch.from_numpy(img).float().permute(2, 0, 1)[None, ] / 255.
    return img

def PSNR(input1, input2):
    mse = torch.mean((input1 - input2) ** 2)
    psnr = 20 * torch.log10(1 / torch.sqrt(mse))
    return psnr.item()


ref_image = to_tensor('/home/ziming_wang/code/UVG/CropVideos/Beauty/Beauty/im032.png').to(device)
input_image = to_tensor('/home/ziming_wang/code/UVG/CropVideos/Beauty/Beauty/im033.png').to(device)

In [ ]:
dpb = {
        "ref_frame": ref_image,
        "ref_feature": None,
        "ref_y": None,
        "ref_mv_y": None,
    }
x = input_image
mv_y_q_scale = p_frame_mv_y_q_scales[rate_idx]
y_q_scale = p_frame_y_q_scales[rate_idx]
curr_mv_y_q = model.get_curr_mv_y_q(mv_y_q_scale)
curr_y_q = model.get_curr_y_q(y_q_scale)

est_mv = model.optic_flow(x, dpb["ref_frame"])
mv_y = model.mv_encoder(est_mv)
mv_y = mv_y / curr_mv_y_q
mv_z = model.mv_hyper_prior_encoder(mv_y)
mv_z_hat = model.quant(mv_z)
mv_params = model.mv_hyper_prior_decoder(mv_z_hat)
ref_mv_y = dpb["ref_mv_y"]
if ref_mv_y is None:
    ref_mv_y = torch.zeros_like(mv_y)
mv_params = torch.cat((mv_params, ref_mv_y), dim=1)
mv_q_step, mv_scales, mv_means = model.mv_y_prior_fusion(mv_params).chunk(3, 1)
mv_y_res, mv_y_q, mv_y_hat, mv_scales_hat = model.forward_prior(
    mv_y, mv_means, mv_scales, mv_q_step, model.mv_y_spatial_prior)
mv_y_hat = mv_y_hat * curr_mv_y_q

mv_hat = model.mv_decoder(mv_y_hat)
context1, context2, context3, warp_frame = model.motion_compensation(dpb, mv_hat)

y = model.contextual_encoder(x, context1, context2, context3)
y = y / curr_y_q
z = model.contextual_hyper_prior_encoder(y)
z_hat = model.quant(z)
hierarchical_params = model.contextual_hyper_prior_decoder(z_hat)
temporal_params = model.temporal_prior_encoder(context3)

ref_y = dpb["ref_y"]
if ref_y is None:
    ref_y = torch.zeros_like(y)
params = torch.cat((temporal_params, hierarchical_params, ref_y), dim=1)
q_step, scales, means = model.y_prior_fusion(params).chunk(3, 1)
y_res, y_q, y_hat, scales_hat = model.forward_prior(
    y, means, scales, q_step, model.y_spatial_prior)
y_hat = y_hat * curr_y_q


In [ ]:
y_hat_ = torch.zeros_like(y_hat)
# y_hat_[0, 52] = y_hat[0, 52]
context1_ = torch.zeros_like(context1)
context2_ = torch.zeros_like(context2)
context3_ = torch.zeros_like(context3)

In [ ]:
recon_image_feature = model.contextual_decoder(y_hat, context2, context3)
feature, recon_image = model.recon_generation_net(recon_image_feature, context1)

In [ ]:
import matplotlib.pyplot as plt
from src.utils.flowlib import flow_to_image
flow_img, _ = flow_to_image(mv_hat[0].detach().cpu().numpy())
plt.imshow(recon_image[0].detach().permute(1, 2, 0).cpu().numpy())
plt.show()
plt.imshow(flow_img)
PSNR(recon_image, input_image)

In [ ]:
y_hat.abs().square().sum(dim=-1).sum(dim=-1)[0].sort()

In [ ]:
x1 = y_res.detach().numpy()
plt.clf()
for i in range(x1.shape[1]):
    plt.figure(i)
    t = x1[0][i]
    t = (t - t.min()) / (t.max() - t.min())
#     plt.matshow(t, aspect='equal', cmap='gray')
    plt.colorbar()
#     plt.show()

In [ ]:
y_res.shape

In [ ]:
# plt.plot(list(range(64)), filter_attention[0, :, 0, 0].detach().numpy())
plt.imshow(spatial_attention[0,0,0,0].detach().numpy())
plt.colorbar()

In [ ]:
from dev.scale_hyper_prior import ScaleHyperprior
import torch
from skimage import io
import torchvision
import numpy as np
import matplotlib.pyplot as plt

def PSNR(input1, input2):
    mse = torch.mean((input1 - input2) ** 2)
    psnr = 20 * torch.log10(1 / torch.sqrt(mse))
    return psnr.item()
# ckpt_path = "/data/lqh/.cache/torch/hub/checkpoints/bmshj2018-hyperprior-8-a583f0cf.pth.tar"
ckpt_path = "scale_hyper_prior.pth"
state_dict = torch.load(ckpt_path, map_location='cpu')
model = ScaleHyperprior.from_state_dict(state_dict)
model.eval()
img = io.imread("kodim23.png")[None,]
img_ = io.imread("kodim23_.png")[None,]
imgs = np.concatenate([img, img_], axis=0)
imgs = torch.from_numpy(imgs).permute(0, 3, 1, 2) / 255.
# res = model.compress(imgs)
# res = model.decompress(res['strings'], res['shape'])
# x_hat =res['x_hat']
# psnr = PSNR(x_hat, img)
# torchvision.utils.save_image(x_hat[0], "kodim23_.png")
# print("PSNR: {:.2f}".format(psnr))

In [ ]:
y = model.g_a(imgs)

In [ ]:
plt.clf()
hist = torch.histc(y, min=-4, max=4, bins=512)
plt.plot(np.arange(-4, 4, 8 / 512.), hist.detach().numpy())
plt.show()

In [ ]:
y_ = y.round()
plt.clf()
hist = torch.histc(y_, min=-4, max=4, bins=512)
plt.plot(np.arange(-4, 4, 8 / 512.), hist.detach().numpy())
plt.show()

# 二、 初始化权重模块

In [ ]:
import torch
from src.models.video_model import VideoCompressor

model = VideoCompressor(use_sal=True)
# torch.save(model.state_dict(), "checkpoints/offset_model_init.pth.tar")
# new_state_dict = torch.load('output/codec-09-24_11-22-24/epoch_0000_step002000.pt', map_location='cpu')
# state_dict = torch.load('checkpoints/acmmm2022_video_psnr.pth.tar', map_location='cpu')
# new_state_dict_ = {}
# for name,param in new_state_dict.items():
#     new_state_dict_[name.replace('module.', '')] = param
# for name, param in new_state_dict_.items():
#     if name in state_dict and param.shape == state_dict[name].shape:
#         new_state_dict_[name] = state_dict[name]
#     else:
#         new_state_dict_[name] = param

# model.load_state_dict(new_state_dict_, strict=True)

state_dict = torch.load('./codec/output/variable-rate-mask/epoch_0006.pt', map_location='cpu')
state_dict_ = {}
for name,param in state_dict.items():
    state_dict_[name.replace('module.', '')] = param

new_state_dict = {}
for name, param in model.named_parameters():
    if name in state_dict_ and param.shape == state_dict_[name].shape:
        new_state_dict[name] = state_dict_[name]
    else:
        new_state_dict[name] = param
        print('not found', name)
torch.save(new_state_dict, "checkpoints_init.pth.tar")

In [ ]:
for name, param in model.named_parameters():
    if 'contextual_condition_prior' in name:
        tar_name = name.replace('condition', 'hyper')
        if param.shape == state_dict_[tar_name].shape:
            new_state_dict[name] = state_dict_[tar_name]
            print('Copy from ', tar_name, 'to', name)
        else:
            print(name)

In [ ]:
torch.save(new_state_dict, "cdconv_checkpoints_init.pth.tar")

# 三、灰度共生矩阵等纹理可视化

In [ ]:
from codec.src.dataset.fast_glcm import gen_features
import imageio.v2 as imageio
from codec.src.dataset.fast_glcm import gen_feature

img = imageio.imread('/data/lqh/deepvideo/SalCodec/kodim23.png')
R = img[:, :, 0]
res = gen_feature(R)
import matplotlib.pyplot as plt

plt.clf()
plt.imshow(R)

# 四、UVG Test Result

In [ ]:
import json
import numpy as np
from collections import defaultdict
import matplotlib.pyplot as plt

def cal_metric(json_path):
    with open(json_path, 'r') as f:
        data = json.load(f)
    bpp = defaultdict()
    psnr = defaultdict()
    missim = defaultdict()
    for ds, results in data.items():
        for seq_name, seq in results.items():
            for rate_num, res in seq.items():
                bpp.setdefault(rate_num, []).append(res['ave_all_frame_bpp'])
                psnr.setdefault(rate_num, []).append(res['ave_all_frame_psnr'])
                missim.setdefault(rate_num, []).append(res['ave_all_frame_msssim'])
    res = defaultdict()
    for rate_num in bpp.keys():
        res.setdefault('bpp', []).append(np.mean(bpp[rate_num]))
        res.setdefault('psnr', []).append(np.mean(psnr[rate_num]))
        res.setdefault('msssim', []).append(np.mean(missim[rate_num]))
    for key, value in res.items():
        res[key] = np.array(value)
    return res


### 单个视频指标

In [ ]:
json_path = '/data/lqh/deepvideo/SalCodec/codec/metric/baseline.json'
with open(json_path, 'r') as f:
    data = json.load(f)
for ds, results in data.items():
    for seq_name, info in results.items():
        bpp, psnr, msssim = [], [], []
        for rate_num, res in info.items():
            bpp.append(res['ave_all_frame_bpp'])
            psnr.append(res['ave_all_frame_psnr'])
            msssim.append(res['ave_all_frame_msssim'])
        plt.plot(bpp, psnr, label=seq_name)
plt.legend()
plt.show()

## 4.1 添加offset部分权重

In [ ]:
baseline = cal_metric('/data/lqh/deepvideo/SalCodec/codec/metric/baseline.json')
new_model = cal_metric('/data/lqh/deepvideo/SalCodec/codec/2step1.json')

plt.clf()
plt.plot(baseline['bpp'], baseline['psnr'], label='baseline')
plt.plot(new_model['bpp'], new_model['psnr'], label='new_model')
plt.legend()
plt.show()

### 交叉weight，减去offset bpp
从曲线上观察，如果offset不算进bpp里面，添加offset还是能轻微提升模型效果

In [ ]:
baseline = cal_metric('/data/lqh/deepvideo/SalCodec/codec/metric/baseline.json')
new_model = cal_metric('/data/lqh/deepvideo/SalCodec/codec/metric/cross_weight_no_offset.json')

plt.clf()
plt.plot(baseline['bpp'], baseline['psnr'], label='baseline')
plt.plot(new_model['bpp'], new_model['psnr'], label='new_model')
plt.legend()
plt.show()

In [ ]:
baseline = cal_metric('/data/lqh/deepvideo/SalCodec/codec/metric/baseline.json')
new_model = cal_metric('/data/lqh/deepvideo/SalCodec/codec/metric/multi_frame_e1s1w.json')

plt.clf()
plt.plot(baseline['bpp'], baseline['psnr'], label='baseline')
plt.plot(new_model['bpp'], new_model['psnr'], label='new_model')
plt.legend()
plt.show()

# 可视化重建帧

In [ ]:
from skimage import io
from skimage.color import rgb2gray
import numpy as np
import matplotlib.pyplot as plt

org_img = io.imread('/data/lqh/deepvideo/DVC/DVC/data/UVG/CropVideos/ShakeNDry/ShakeNDry/im001.png')
dec_img = io.imread('/data/lqh/deepvideo/DVC/DVC/data/UVG/CropVideos/Bosphorus/Bosphorus/im001.png')
h, w = org_img.shape[:-1]
plt.imshow(dec_img)
plt.show()
psnr = PSNR(org_img, dec_img)
gray = rgb2gray(dec_img)
plt.imshow(gray, 'gray')
plt.show()
f = np.fft.fft2(gray)
f = np.fft.fftshift(f)
plt.imshow(np.log(np.abs(f)))
plt.show()

gray = rgb2gray(org_img)
plt.imshow(gray, 'gray')
# plt.show()
f = np.fft.fft2(gray)
f = np.fft.fftshift(f)
plt.imshow(np.log(np.abs(f)))
# plt.show()

In [ ]:
d = {
    'Beauty':'/data/lqh/deepvideo/DVC/DVC/data/UVG/CropVideos/Beauty/Beauty/im002.png',
    'Bosphorus':'/data/lqh/deepvideo/DVC/DVC/data/UVG/CropVideos/Bosphorus/Bosphorus/im002.png',
    'HoneyBee':'/data/lqh/deepvideo/DVC/DVC/data/UVG/CropVideos/HoneyBee/HoneyBee/im002.png',
    'ReadySteadyGo':'/data/lqh/deepvideo/DVC/DVC/data/UVG/CropVideos/ReadySteadyGo/ReadySteadyGo/im002.png',
    'ShakeNDry':'/data/lqh/deepvideo/DVC/DVC/data/UVG/CropVideos/ShakeNDry/ShakeNDry/im002.png',
    'Jockey':'/data/lqh/deepvideo/DVC/DVC/data/UVG/CropVideos/Jockey/Jockey/im002.png',
    'YachtRide':'/data/lqh/deepvideo/DVC/DVC/data/UVG/CropVideos/YachtRide/YachtRide/im002.png'
}
plt.clf()
for name, path  in d.items():
    img = io.imread(path)
    hist = np.histogram(img, bins=25)
    plt.plot(hist[1][1:], hist[0], label=name)
plt.legend()
plt.show()

In [ ]:
org_img = io.imread('/data/lqh/deepvideo/DVC/DVC/data/UVG/CropVideos/Bosphorus/Bosphorus/im001.png')
gray = rgb2gray(org_img)
plt.imshow(gray, 'gray')
plt.clf()
hist = np.histogram(gray, bins=25)
plt.plot(hist[1][1:], hist[0], label=name)

hist = np.histogram(gray ** 0.45, bins=25)
plt.plot(hist[1][1:], hist[0], label=name)
plt.show()

In [ ]:
import json
import numpy as np

def extract_hard_case(json_path, seq_len=4000):
    with open(json_path, 'r') as f:
        data = json.load(f)
    res = []
    for k, v in data.items():
        seqs = data[k]
        seqs = sorted(seqs.items(), key=lambda x: x[1], reverse=True)
        res.extend(seqs[:seq_len])
    res_ = []
    for k, v in res:
        res_.append('/'.join(k.split('/')[-3:][:-1]))
    return list(set(res_))

json_path = ['output/codec-10-15_10-43-41/epoch_0000.json',
             'output/codec-10-15_10-43-41/epoch_0001.json',
             'output/codec-10-15_10-43-41/epoch_0002.json',
             'output/codec-10-15_10-43-41/epoch_0003.json',
             'output/codec-10-15_10-43-41/epoch_0004.json',
             'output/codec-10-15_10-43-41/epoch_0005.json',
             'output/codec-10-15_10-43-41/epoch_0006.json',
            ]

seq_len = 4000
hard_case  =[]
for js in json_path:
    hard_case.extend(extract_hard_case(js, seq_len))
hard_case = list(set(hard_case))
np.savetxt("hard_case.txt", hard_case, fmt='%s', delimiter='\n')
print(len(hard_case))

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

max_val = 2.2
min_val = 1.0
num = 7
values = np.linspace(np.log(max_val), np.log(min_val), num)
values = np.exp(values)
plt.plot(list(range(len(values))), values)
print(', '.join(list(map(str, np.around(values, 4).tolist()))))